Cuaderno de entrenamiento

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import warnings
warnings.filterwarnings('ignore')

# Cargar datos
print("=== Cargando datos ===")

from google.colab import drive
drive.mount("/content/drive")


df_proc = pd.read_csv("/content/drive/MyDrive/DeepLearning/House_Rent_Dataset.csv")

# Filtrar registros con categorías poco frecuentes
df_proc = df_proc[(df_proc['Area Type'] != 'Built Area') &
                  (df_proc['Point of Contact'] != 'Contact Builder')]

print(f"Dimensiones del dataset después de filtrado: {df_proc.shape}")

# Extraer información de Floor
def extract_floor_info(floor_str):
    """
    Extrae información del piso y total de pisos.
    Retorna: (número_de_piso, total_de_pisos)
    """
    try:
        floor_str = str(floor_str).strip()

        # Procesar valores especiales
        if "Upper Basement" in floor_str:
            floor_num = -1
        elif "Lower Basement" in floor_str:
            floor_num = -2
        elif "Ground" in floor_str:
            floor_num = 0
        else:
            # Extraer número de piso
            parts = floor_str.split("out of")
            if len(parts) > 0 and parts[0].strip().isdigit():
                floor_num = int(parts[0].strip())
            else:
                floor_num = None

        # Extraer total de pisos
        if "out of" in floor_str:
            parts = floor_str.split("out of")
            if len(parts) > 1 and parts[1].strip().isdigit():
                total_floors = int(parts[1].strip())
            else:
                total_floors = None
        else:
            total_floors = None

        return floor_num, total_floors
    except:
        return None, None

# Aplicar extracción
print("=== Procesando columna Floor ===")
df_proc[['Floor_Number', 'Total_Floors']] = df_proc['Floor'].apply(
    lambda x: pd.Series(extract_floor_info(x))
)

# Calcular ratio (con manejo de casos nulos)
df_proc['Floor_Ratio'] = None
mask = (~df_proc['Floor_Number'].isna()) & (~df_proc['Total_Floors'].isna()) & (df_proc['Total_Floors'] > 0)
df_proc.loc[mask, 'Floor_Ratio'] = df_proc.loc[mask, 'Floor_Number'] / df_proc.loc[mask, 'Total_Floors']

# Rellenar valores nulos con la mediana
df_proc['Floor_Number'] = df_proc['Floor_Number'].fillna(df_proc['Floor_Number'].median())
df_proc['Total_Floors'] = df_proc['Total_Floors'].fillna(df_proc['Total_Floors'].median())
df_proc['Floor_Ratio'] = df_proc['Floor_Ratio'].fillna(df_proc['Floor_Ratio'].median())

# Aplicar transformación logarítmica
print("=== Aplicando transformación logarítmica ===")
df_proc['Rent_log'] = np.log(df_proc['Rent'])
df_proc['Size_log'] = np.log(df_proc['Size'])

# Eliminar columnas que no aportan valor predictivo
columnas_a_eliminar = ['Posted On', 'Area Locality', 'Floor']
X = df_proc.drop(columns=columnas_a_eliminar + ['Rent', 'Rent_log']).copy()
y = df_proc['Rent_log'].copy()

# Guardar los valores originales para evaluación
y_original = df_proc['Rent'].copy()

print("Columnas después de eliminaciones:")
print(X.columns.tolist())

# Verificar valores nulos restantes
print("\nValores nulos por columna en X:")
print(X.isna().sum())

# Rellenar valores nulos restantes
for col in X.columns:
    if X[col].isna().any():
        if X[col].dtype.kind in 'ifc':  # Numérico
            X[col] = X[col].fillna(X[col].median())
        else:  # Categórico
            X[col] = X[col].fillna(X[col].mode()[0])

# Verificar que no quedan valores nulos
print("\nValores nulos después de imputación:")
print(X.isna().sum().sum())

# Identificar tipos de columnas
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\nColumnas categóricas ({len(cat_cols)}):")
print(cat_cols)
print(f"\nColumnas numéricas ({len(num_cols)}):")
print(num_cols)

# División en train/test (80% train, 20% test)
X_train, X_test, y_train, y_test, y_train_original, y_test_original = train_test_split(
    X, y, y_original, test_size=0.2, random_state=42
)

# Crear preprocesador (con sparse_output=False para OneHotEncoder)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols)
    ]
)

# Aplicar preprocesamiento
preprocessor.fit(X_train)
X_train_prep = preprocessor.transform(X_train)
X_test_prep = preprocessor.transform(X_test)

print(f"\nDatos preprocesados:")
print(f"X_train_prep: {X_train_prep.shape}")
print(f"X_test_prep: {X_test_prep.shape}")

# Verificar valores NaN en los datos preprocesados
print(f"NaN en X_train_prep: {np.isnan(X_train_prep).any()}")
print(f"NaN en X_test_prep: {np.isnan(X_test_prep).any()}")

# Reemplazar NaN en datos preprocesados si hay alguno
if np.isnan(X_train_prep).any():
    print("Reemplazando NaN en datos preprocesados...")
    X_train_prep = np.nan_to_num(X_train_prep)
    X_test_prep = np.nan_to_num(X_test_prep)

# Definir función para crear modelos
def crear_modelo(config):
    """
    Crea un modelo de red neuronal con la configuración especificada
    """
    model = Sequential()

    # Primera capa (entrada)
    model.add(Dense(
        config['hidden_layers'][0],
        activation=config['activation'],
        input_shape=(X_train_prep.shape[1],),
        kernel_regularizer=None if config.get('l2_reg') is None else tf.keras.regularizers.l2(config.get('l2_reg'))
    ))

    if config.get('batch_norm', False):
        model.add(BatchNormalization())

    if config.get('dropout', 0) > 0:
        model.add(Dropout(config['dropout']))

    # Capas ocultas
    for units in config['hidden_layers'][1:]:
        model.add(Dense(
            units,
            activation=config['activation'],
            kernel_regularizer=None if config.get('l2_reg') is None else tf.keras.regularizers.l2(config.get('l2_reg'))
        ))

        if config.get('batch_norm', False):
            model.add(BatchNormalization())

        if config.get('dropout', 0) > 0:
            model.add(Dropout(config['dropout']))

    # Capa de salida
    model.add(Dense(1))

    # Compilar modelo
    if config.get('optimizer', 'adam') == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=config.get('learning_rate', 0.001))
    else:
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=config.get('learning_rate', 0.001))

    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    return model

# Definir experimentos
print("\n=== Definiendo experimentos ===")
experimentos = [
    {
        'name': "Dropout",
        'hidden_layers': [128, 64, 32],
        'activation': 'relu',
        'dropout': 0.2,
        'learning_rate': 0.001,
        'batch_size': 32,
        'epochs': 200
    },
    {
        'name': "L2",
        'hidden_layers': [128, 64, 32],
        'activation': 'relu',
        'l2_reg': 0.01,
        'learning_rate': 0.001,
        'batch_size': 32,
        'epochs': 200
    },
    {
        'name': "Combined",
        'hidden_layers': [128, 64, 32],
        'activation': 'relu',
        'dropout': 0.2,
        'l2_reg': 0.01,
        'batch_norm': True,
        'learning_rate': 0.001,
        'batch_size': 32,
        'epochs': 200
    }
]

# Función para evaluar modelos (simplificada con transformación logarítmica)
def evaluar_modelo(modelo, X_train, y_train, X_test, y_test, y_train_original, y_test_original):
    """
    Evalúa un modelo y devuelve métricas en escalas transformada y original
    """
    # Predicciones en escala logarítmica
    y_train_pred_log = modelo.predict(X_train).flatten()
    y_test_pred_log = modelo.predict(X_test).flatten()

    # Métricas en escala logarítmica
    train_mse_log = mean_squared_error(y_train, y_train_pred_log)
    test_mse_log = mean_squared_error(y_test, y_test_pred_log)
    train_mae_log = mean_absolute_error(y_train, y_train_pred_log)
    test_mae_log = mean_absolute_error(y_test, y_test_pred_log)
    train_r2_log = r2_score(y_train, y_train_pred_log)
    test_r2_log = r2_score(y_test, y_test_pred_log)

    # Transformar a escala original
    y_train_pred_original = np.exp(y_train_pred_log)
    y_test_pred_original = np.exp(y_test_pred_log)

    # Métricas en escala original
    train_mse = mean_squared_error(y_train_original, y_train_pred_original)
    test_mse = mean_squared_error(y_test_original, y_test_pred_original)
    train_mae = mean_absolute_error(y_train_original, y_train_pred_original)
    test_mae = mean_absolute_error(y_test_original, y_test_pred_original)
    train_r2 = r2_score(y_train_original, y_train_pred_original)
    test_r2 = r2_score(y_test_original, y_test_pred_original)

    # MAPE (error porcentual medio absoluto)
    train_mape = np.mean(np.abs((y_train_original - y_train_pred_original) / y_train_original)) * 100
    test_mape = np.mean(np.abs((y_test_original - y_test_pred_original) / y_test_original)) * 100

    # Métricas resumidas
    metricas = {
        'train_mse_log': train_mse_log,
        'test_mse_log': test_mse_log,
        'train_mae_log': train_mae_log,
        'test_mae_log': test_mae_log,
        'train_r2_log': train_r2_log,
        'test_r2_log': test_r2_log,
        'train_mse': train_mse,
        'test_mse': test_mse,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'train_mape': train_mape,
        'test_mape': test_mape
    }

    return metricas

# Entrenar y evaluar cada modelo
resultados = []
tiempo_inicio_total = time.time()

for config in experimentos:
    print(f"\n=== Entrenando modelo: {config['name']} ===")
    tiempo_inicio = time.time()

    # Crear modelo
    modelo = crear_modelo(config)

    # Callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=30,
        restore_best_weights=True,
        verbose=1
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=0.0001,
        verbose=1
    )

    # Entrenar modelo
    history = modelo.fit(
        X_train_prep, y_train,
        validation_split=0.2,  # Usar 20% de train como validación
        epochs=config['epochs'],
        batch_size=config['batch_size'],
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    # Evaluar modelo
    metricas = evaluar_modelo(modelo, X_train_prep, y_train, X_test_prep, y_test,
                              y_train_original, y_test_original)

    tiempo_fin = time.time()
    tiempo_entrenamiento = tiempo_fin - tiempo_inicio

    # Guardar resultados
    resultado = {
        'config': config,
        'modelo': modelo,
        'history': history.history,
        'metricas': metricas,
        'tiempo_entrenamiento': tiempo_entrenamiento
    }
    resultados.append(resultado)

    # Mostrar métricas principales
    print(f"Tiempo de entrenamiento: {tiempo_entrenamiento:.2f} segundos")
    print(f"MSE (test): {metricas['test_mse']:.2f}")
    print(f"MAE (test): {metricas['test_mae']:.2f}")
    print(f"MAPE (test): {metricas['test_mape']:.2f}%")
    print(f"R² (test): {metricas['test_r2']:.4f}")

tiempo_fin_total = time.time()
tiempo_total = tiempo_fin_total - tiempo_inicio_total
print(f"\nTiempo total de experimentación: {tiempo_total/60:.2f} minutos")

# Crear DataFrame con resumen de resultados
df_resultados = pd.DataFrame([
    {
        'Modelo': res['config']['name'],
        'Capas': str(res['config']['hidden_layers']),
        'Activación': res['config']['activation'],
        'Dropout': res['config'].get('dropout', 0),
        'L2': res['config'].get('l2_reg', 0),
        'BatchNorm': res['config'].get('batch_norm', False),
        'MSE': res['metricas']['test_mse'],
        'MAE': res['metricas']['test_mae'],
        'MAPE (%)': res['metricas']['test_mape'],
        'R²': res['metricas']['test_r2'],
        'Tiempo (s)': res['tiempo_entrenamiento']
    } for res in resultados
])

# Mostrar resultados ordenados por R²
print("\n=== Resumen de resultados ===")
display(df_resultados.sort_values('R²', ascending=False))

# Visualizar comparación de pérdida de validación
plt.figure(figsize=(12, 6))
for res in resultados:
    plt.plot(res['history']['val_loss'], label=res['config']['name'])
plt.title('Comparación de Pérdida de Validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida (MSE)')
plt.legend()
plt.grid(True)
plt.show()

# Obtener mejor modelo
mejor_idx = df_resultados['R²'].idxmax()
mejor_modelo = resultados[mejor_idx]['modelo']
mejor_config = resultados[mejor_idx]['config']

print(f"\n=== Mejor modelo: {mejor_config['name']} ===")
print(f"R² (test): {df_resultados.iloc[mejor_idx]['R²']:.4f}")
print(f"MAPE (test): {df_resultados.iloc[mejor_idx]['MAPE (%)']:.2f}%")

# Visualizar predicciones vs valores reales para el mejor modelo
y_pred_log = mejor_modelo.predict(X_test_prep).flatten()
y_pred_original = np.exp(y_pred_log)

plt.figure(figsize=(10, 6))
plt.scatter(y_test_original, y_pred_original, alpha=0.5)
plt.plot([y_test_original.min(), y_test_original.max()],
         [y_test_original.min(), y_test_original.max()], 'r--')
plt.title('Predicciones vs Valores Reales')
plt.xlabel('Precio Real')
plt.ylabel('Precio Predicho')
plt.grid(True)
plt.show()

# Histograma de errores
errores = y_test_original - y_pred_original
plt.figure(figsize=(10, 6))
plt.hist(errores, bins=50)
plt.title('Distribución de Errores')
plt.xlabel('Error de Predicción')
plt.ylabel('Frecuencia')
plt.grid(True, alpha=0.3)
plt.show()

# Guardar el mejor modelo
mejor_modelo.save('mejor_modelov2.h5')
import joblib
joblib.dump(preprocessor, 'preprocessor.joblib')

print("\n=== Modelo y preprocesador guardados ===")
print("Modelo guardado como 'mejor_modelo.h5'")
print("Preprocesador guardado como 'preprocessor.joblib'")

# Análisis por ciudad
print("\n=== Análisis por ciudad ===")
# Combinar predicciones con datos originales para análisis
test_results = pd.DataFrame({
    'Ciudad': X_test['City'].values,
    'Precio_Real': y_test_original.values,
    'Precio_Predicho': y_pred_original,
    'Error': y_test_original.values - y_pred_original,
    'Error_Porcentual': ((y_test_original.values - y_pred_original) / y_test_original.values) * 100
})

# Análisis por ciudad
city_analysis = test_results.groupby('Ciudad').agg({
    'Precio_Real': 'mean',
    'Precio_Predicho': 'mean',
    'Error': ['mean', 'std'],
    'Error_Porcentual': ['mean', 'std']
}).round(2)

display(city_analysis)

# Visualizar rendimiento por ciudad
plt.figure(figsize=(12, 6))
city_mape = test_results.groupby('Ciudad')['Error_Porcentual'].apply(lambda x: np.mean(np.abs(x)))
city_mape.sort_values().plot(kind='bar')
plt.title('Error Porcentual Absoluto Medio (MAPE) por Ciudad')
plt.xlabel('Ciudad')
plt.ylabel('MAPE (%)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Guardar resumen de resultados
df_resultados.to_csv('resultados_experimentos.csv', index=False)
print("Resumen de resultados guardado como 'resultados_experimentos.csv'")

Hola Mundo
